In [29]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import statistics
import tensorflow as tf
import re
import os

In [8]:
from tensorflow.keras.layers import MultiHeadAttention, Embedding

In [50]:
# os.path.joi
fall = os.path.join(os.getcwd() , 'datasets/Fall/***.xlsx')
adl = os.path.join(os.getcwd(), 'datasets/ADL/***.xlsx')

In [51]:
fall_files = glob.glob(fall)
adl_files = glob.glob(adl)

In [52]:
adl_files

['/Users/tousif/Lstm_transformer/KD_Multimodal/datasets/ADL/Kseat_Hip_2022-08-04T17.22.00.826_E59CF70AD2E7_Accelerometer_32.000Hz_1.7.3.xlsx',
 '/Users/tousif/Lstm_transformer/KD_Multimodal/datasets/ADL/S wash_Right_2022-07-21T09.40.55.532_DE52EF1F2A06_Accelerometer_32.000Hz_1.7.3.xlsx',
 '/Users/tousif/Lstm_transformer/KD_Multimodal/datasets/ADL/Rpick1_Right_2022-07-30T17.16.13.176_DE52EF1F2A06_Accelerometer_32.000Hz_1.7.3.xlsx',
 '/Users/tousif/Lstm_transformer/KD_Multimodal/datasets/ADL/S walk_Hip_2022-07-21T09.13.42.948_E59CF70AD2E7_Accelerometer_32.000Hz_1.7.3.xlsx',
 '/Users/tousif/Lstm_transformer/KD_Multimodal/datasets/ADL/W put1_Hip_2022-07-27T11.48.32.387_E59CF70AD2E7_Accelerometer_32.000Hz_1.7.3.xlsx',
 '/Users/tousif/Lstm_transformer/KD_Multimodal/datasets/ADL/G walk_Right_2022-07-26T14.17.23.635_DE52EF1F2A06_Accelerometer_32.000Hz_1.7.3.xlsx',
 '/Users/tousif/Lstm_transformer/KD_Multimodal/datasets/ADL/Kput2_Hip_2022-08-04T18.06.39.182_E59CF70AD2E7_Accelerometer_32.000Hz_1

In [12]:
all_file_path = fall_files + adl_files

In [21]:
file_len = []
trials = []
labels = []
fall_pattern = re.compile("Fall")
for file_path in all_file_path:
    
    #checking if the excel has 2 sheets or not
    label = None
    if fall_pattern.search(file_path):
        label = 1
    else:
        label = 0
                 
    if len (pd.ExcelFile(file_path).sheet_names) == 2:
        fall_df = pd.read_excel(file_path, sheet_name = -1)
        df = pd.read_excel(fall_files[1], sheet_name=-1)
        df = df.iloc[:, :-1]
        null_col = df[df.isnull().any(axis = 1)].index.to_list()
        
        #calculating how many null segments we have 
        null_col = df[df.isnull().any(axis = 1)].index.to_list()
        null_seg = len(null_col)//10
        trial_start_lst = null_col[9::10]
        trial_end_lst = null_col[10::10]
        
        for i in range(len(null_col)//10 + 1):
            trial = None
            if i == 0 :
                trial = df.iloc[0:null_col[1]-1, 3:6]
            elif i == null_seg :
                trial = df.iloc[trial_start_lst[-1]+1:, 3:6]
            else: 
                trial_end = trial_end_lst[i-1]
                trial_start = trial_start_lst[i-1] + 1
                trial = df.iloc[trial_start : trial_end-1 , 3:6].values
                trial = tf.convert_to_tensor(trial, dtype = tf.float32)
            labels.append(label)
            trials.append(trial)
trials = tf.keras.utils.pad_sequences(trials, maxlen= 512, value = float('-inf') , dtype = float, padding = 'post' )

#transposing the trials 
trials = tf.transpose(trials, perm = [0,2,1])


Metal device set to: Apple M1 Pro


2022-11-20 17:28:11.875107: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-20 17:28:11.875737: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [24]:
#Git commit
!git add .
!git commit -m "Feat: Creating the Full Dataset"
!git push 



[main 8b74328] Feat: Creating the Full Dataset
 4 files changed, 5356 insertions(+), 156087 deletions(-)
 create mode 100644 raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/FordA_TEST.tsv
 create mode 100644 raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/FordA_TRAIN.tsv
Enumerating objects: 16, done.
Counting objects: 100% (16/16), done.
Delta compression using up to 8 threads
Compressing objects: 100% (8/8), done.
Writing objects: 100% (12/12), 11.12 MiB | 1.78 MiB/s, done.
Total 12 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 1 local object.
To https://github.com/tousifulhaque/KD_Multimodal.git
   c23d055..8b74328  main -> main


In [56]:
masking_layer = tf.keras.layers.Masking(mask_value = float('-inf'), input_shape = (512, 3))

In [53]:
masking_layer = tf.keras.layers.Embedding(input_dim = 512, output_dim = 64)

In [27]:
np.savez_compressed("fall_detection_dataset", trials=trials, labels=labels)